### Chapter 9: 2D axissymmetric isoparametric tri element

Additionally: Dr. Szekrényes András, Chapter 13 for the quadratic element.

In [2]:
import numpy as np
import sympy as sp
sp.init_printing()

### The 2D axissymmetric tri element

This is a triangular element with nodes: i, j, m.

Coordinates are: r, z for radial and vertical directions, respectively.

Each node hat 2 degrees of freedom (DOFs):
- u: radial displacement
- w: vertical displacement

The DOFs are ordered as follows: u, w.

### The physical properties of the element

In [3]:
ND = 2  # number of degrees of freedom per node
NNODE = 4  # number of nodes
if NNODE == 3:
    NGS = [1, 3]  # number of Gauss points, can be 1, 3
elif NNODE == 4:
    NGS = [1, 4]  # number of Gauss points, can be 1, 2, 3, 4, 5, or 6
else:
    raise ValueError("Unsupported number of nodes. Only 3 or 4 nodes are supported")


In [4]:

### The physical properties of the element
E = 200e9  # Young's modulus in Pa
nu = 0.3  # Poisson's ratio

# Define symbolic physical coordinates
r1, z1, r2, z2, r3, z3, r4, z4 = sp.symbols('r1 z1 r2 z2 r3 z3 r4 z4')

# Define symbolic variables. Capital letters R, Z are the symbols for the variables r, z
R, Z = sp.symbols('R Z')  # radial and vertical coordinates

if NNODE == 3:
    # Example coordinates for the nodes
    # physical_coords = {'P1': (0, 0),
    #                    'P2': (1, 0),
    #                    'P3': (0, 1)}  # example coordinates for the nodes
    physical_coords = {'P1': (3, 0),
                       'P2': (4, 0),
                       'P3': (3, 1)}  # example coordinates for the nodes
    # physical_coords = {'P1': (0, 0),
    #                    'P2': (3, 0),
    #                    'P3': (0, 2)}  # example coordinates for the nodes

    midpoint = (physical_coords['P1'][0] + physical_coords['P2'][0] + physical_coords['P3'][0]) / 3, \
                (physical_coords['P1'][1] + physical_coords['P2'][1] + physical_coords['P3'][1]) / 3

    physical_coords_map = {'r1': physical_coords['P1'][0], 'z1': physical_coords['P1'][1],
                           'r2': physical_coords['P2'][0], 'z2': physical_coords['P2'][1],
                           'r3': physical_coords['P3'][0], 'z3': physical_coords['P3'][1]}
    # Define the physical nodes as a matrix so it can be used a symbolic evaluation
    physical_nodes = sp.Matrix([[r1, z1],
                                [r2, z2],
                                [r3, z3]])


elif NNODE == 4:
    # Example coordin
    # physical_coords = {'P1': (0.02, 0),
    #                    'P2': (0.11, 0),
    #                    'P3': (0.11, 0.04),
    #                    'P4': (0.02, 0.04),
    #                    }  # example coordinates for the nodesates for the nodes
    physical_coords = {'P1': (0.11, 0),
                       'P2': (0.20, 0),
                       'P3': (0.20, 0.04),
                       'P4': (0.11, 0.04),
                       }  # example coordinates for the nodesates for the nodes

    midpoint = (physical_coords['P1'][0] + physical_coords['P2'][0] + physical_coords['P3'][0] + physical_coords['P4'][0]) / 4, \
                (physical_coords['P1'][1] + physical_coords['P2'][1] + physical_coords['P3'][1] + physical_coords['P4'][1]) / 4

    physical_coords_map = {'r1': physical_coords['P1'][0], 'z1': physical_coords['P1'][1],
                           'r2': physical_coords['P2'][0], 'z2': physical_coords['P2'][1],
                           'r3': physical_coords['P3'][0], 'z3': physical_coords['P3'][1],
                           'r4': physical_coords['P4'][0], 'z4': physical_coords['P4'][1]
                           }
    # Define the physical nodes as a matrix so it can be used a symbolic evaluation
    physical_nodes = sp.Matrix([[r1, z1],
                                [r2, z2],
                                [r3, z3],
                                [r4, z4],
                                ])


else:
    raise ValueError("Unsupported number of nodes. Only 3 or 4 nodes are supported")

### The isoparametric part

In [5]:
# coordinates of the nodes in the element
n1 = np.array([0, 0])  # node 1 coordinates
n2 = np.array([1, 0])   # node 2 coordinates
n3 = np.array([0, 1])    # node 3 coordinates
n4 = np.array([1, 1])    # node 4 coordinates
if NNODE == 3:
    natural_nodes = np.array([n1, n2, n3])  # all nodes
elif NNODE == 4:
    natural_nodes = np.array([n1, n2, n3, n4])  # all nodes

### Shape functions and their derivatives

In [6]:
# Define the shape functions symbolically
def shape_functions() -> sp.Matrix:
    """shape functions"""

    if NNODE == 3:
        return sp.Matrix([
            1 - R - Z,
            R,
            Z,
        ])
    elif NNODE == 4:
        return sp.Matrix([
            0.25 * (1 - R) * (1 - Z),
            0.25 * (1 + R) * (1 - Z),
            0.25 * (1 + R) * (1 + Z),
            0.25 * (1 - R) * (1 + Z),
        ])
    else:
        raise ValueError("Unsupported number of nodes. Only 3 or 4 nodes are supported")

def isoparametric_mapping() -> np.array:
    """
    Isoparametric mapping natural -> physical coordinates.

    :return: functions returning the physical coordinates
    """

    sh = shape_functions()  # shape functions with substituted values
    return sp.Matrix([sp.Matrix(sh).dot(physical_nodes[:, 0]), sp.Matrix(sh).dot(physical_nodes[:, 1])])

# Calculate the derivatives by s and t, symbolically
def shape_function_derivatives() -> np.array:
    """
    Shape function derivatives.
    [0] = dNi/ds, [1] = dNi/dt
    """
    sh = shape_functions()  # N1, N2, N3, N4
    dN_ds = []
    dN_dt = []

    for i in range(len(sh)):
        dN_ds.append(sp.diff(sh[i], R))
        dN_dt.append(sp.diff(sh[i], Z))

    return dN_ds, dN_dt

print()
print('The shape functions are:')
print(shape_functions())
print()
print('The derivatives are:')
derivatives = shape_function_derivatives()
for i, dN in enumerate(derivatives[0]):
    print(f'dN{i+1}/ds = {dN}')
for i, dN in enumerate(derivatives[1]):
    print(f'dN{i+1}/dt = {dN}')

print()
print('The isoparametric mapping is:')
x_map = isoparametric_mapping()[0]
y_map = isoparametric_mapping()[1]
print(f'x = {x_map}')
print(f'y = {y_map}')

# Substitute the symbolic coordinates with numerical values
st_values = {
    R: 0.3333,  # example value for s
    Z: 0.3333,  # example value for t
}
print()
print('The isoparametric mapping with coordinate values substituted is:')
x_map_substituted = x_map.subs(st_values)
y_map_substituted = y_map.subs(st_values)
print(f'x = {x_map_substituted}')
print(f'y = {y_map_substituted}')

print()
print('The derivatives of the shape functions by s are:')
print(derivatives[0])  # derivatives by s

print()
print('The derivatives of the shape functions by t are:')
print(derivatives[1])  # derivatives by t



The shape functions are:
Matrix([[(0.25 - 0.25*R)*(1 - Z)], [(1 - Z)*(0.25*R + 0.25)], [(0.25*R + 0.25)*(Z + 1)], [(0.25 - 0.25*R)*(Z + 1)]])

The derivatives are:
dN1/ds = 0.25*Z - 0.25
dN2/ds = 0.25 - 0.25*Z
dN3/ds = 0.25*Z + 0.25
dN4/ds = -0.25*Z - 0.25
dN1/dt = 0.25*R - 0.25
dN2/dt = -0.25*R - 0.25
dN3/dt = 0.25*R + 0.25
dN4/dt = 0.25 - 0.25*R

The isoparametric mapping is:
x = r1*(0.25 - 0.25*R)*(1 - Z) + r2*(1 - Z)*(0.25*R + 0.25) + r3*(0.25*R + 0.25)*(Z + 1) + r4*(0.25 - 0.25*R)*(Z + 1)
y = z1*(0.25 - 0.25*R)*(1 - Z) + z2*(1 - Z)*(0.25*R + 0.25) + z3*(0.25*R + 0.25)*(Z + 1) + z4*(0.25 - 0.25*R)*(Z + 1)

The isoparametric mapping with coordinate values substituted is:
x = 0.1111222225*r1 + 0.2222277775*r2 + 0.4444222225*r3 + 0.2222277775*r4
y = 0.1111222225*z1 + 0.2222277775*z2 + 0.4444222225*z3 + 0.2222277775*z4

The derivatives of the shape functions by s are:
[0.25*Z - 0.25, 0.25 - 0.25*Z, 0.25*Z + 0.25, -0.25*Z - 0.25]

The derivatives of the shape functions by t are:
[0.25*

### The shape function matrix

The shape function matrix $N$ is a matrix that contains the shape functions for each node. It has the structure:
$$
N = \begin{bmatrix}
N_i & 0 & N_j & 0 & N_m & 0 \\
0 & N_i & 0 & N_j & 0 & N_m & ...\\
\end{bmatrix}
$$

In [7]:
def N_matrix() -> np.array:
    """
    shape function matrix N.
    It has the structure:
    [[Ni, 0, Nj, 0, Nm, 0],
     [0, Ni, 0, Nj, 0, Nm]]

    """
    _N = shape_functions()  # the shape functions

    # preparing the return matrix
    _ret = sp.Matrix(ND, ND * NNODE, lambda i, j: 0)  # ND rows, ND * NNODE columns

    # filling the matrix
    for i in range(NNODE):
        for j in range(ND):
            _ret[j, j + ND * i] = _N[i]

    return _ret

_N = N_matrix()
print("The shape function matrix N is {}:".format(_N.shape))
print()
print('The shape function matrix N for this element:')
_N_e = _N.subs(physical_coords_map)
sp.pprint(_N_e)

print('\nThe shape function matrix N for this element at node 1:')
_N_e_node1 = _N_e.subs({'R': midpoint[0], 'Z': midpoint[1]})
sp.pprint(_N_e_node1.applyfunc(lambda x: x.round(4)))




The shape function matrix N is (2, 8):

The shape function matrix N for this element:
⎡(0.25 - 0.25⋅R)⋅(1 - Z)             0             (1 - Z)⋅(0.25⋅R + 0.25)     ↪
⎢                                                                              ↪
⎣           0             (0.25 - 0.25⋅R)⋅(1 - Z)             0             (1 ↪

↪          0             (0.25⋅R + 0.25)⋅(Z + 1)             0             (0. ↪
↪                                                                              ↪
↪  - Z)⋅(0.25⋅R + 0.25)             0             (0.25⋅R + 0.25)⋅(Z + 1)      ↪

↪ 25 - 0.25⋅R)⋅(Z + 1)             0           ⎤
↪                                              ⎥
↪         0             (0.25 - 0.25⋅R)⋅(Z + 1)⎦

The shape function matrix N for this element at node 1:
⎡0.207    0    0.283    0    0.2945    0     0.2155    0   ⎤
⎢                                                          ⎥
⎣  0    0.207    0    0.283    0     0.2945    0     0.2155⎦


### Jacobian matrix

In [8]:
def jacobian() -> sp.Matrix:
    """
    Calculate the jacobian matrix for the isoparametric mapping.
    """
    # The isoparametric mapping functions r(R,Z) and z(R,Z)
    mapping = isoparametric_mapping()
    r_map = mapping[0]
    z_map = mapping[1]

    # Calculate the derivatives
    dr_dR = sp.diff(r_map, R)
    dr_dZ = sp.diff(r_map, Z)
    dz_dR = sp.diff(z_map, R)
    dz_dZ = sp.diff(z_map, Z)

    return sp.Matrix([[dr_dR, dr_dZ],
                      [dz_dR, dz_dZ]])

# Calculate the jacobian matrix symbolically
J = jacobian()

# Substitute the symbolic coordinates with numerical values in the Jacobian
J_substituted = J.subs(physical_coords_map)

print('The jacobian matrix with coordinate values substituted is:')
sp.pprint(J_substituted)
print()

# Evaluate the jacobian matrix at some points
for r_eval in [0, 0.3333]:
    for z_eval in [0, 0.3333]:
        J_evaluated = J_substituted.subs({R: r_eval, Z: z_eval})
        print(f'Jacobian matrix at r={r_eval}, z={z_eval}:')
        sp.pprint(J_evaluated)
        print(f'The determinant at this point is |J|={J_evaluated.det():.2f}')
        print()

The jacobian matrix with coordinate values substituted is:
⎡0.045  3.46944695195361e-18⎤
⎢                           ⎥
⎣  0            0.02        ⎦

Jacobian matrix at r=0, z=0:
⎡0.045  3.46944695195361e-18⎤
⎢                           ⎥
⎣  0            0.02        ⎦
The determinant at this point is |J|=0.00

Jacobian matrix at r=0, z=0.3333:
⎡0.045  3.46944695195361e-18⎤
⎢                           ⎥
⎣  0            0.02        ⎦
The determinant at this point is |J|=0.00

Jacobian matrix at r=0.3333, z=0:
⎡0.045  3.46944695195361e-18⎤
⎢                           ⎥
⎣  0            0.02        ⎦
The determinant at this point is |J|=0.00

Jacobian matrix at r=0.3333, z=0.3333:
⎡0.045  3.46944695195361e-18⎤
⎢                           ⎥
⎣  0            0.02        ⎦
The determinant at this point is |J|=0.00



### The $\begin{bmatrix}B\end{bmatrix}$ matrix

In [14]:
def calculate_B():
    """
    Calculates the complete BA-matrix for the element.
    B relates the strain vector [ε_r, ε_θ, ε_z, γ_zr] to the nodal displacement vector.
    """
    J_symbolic = jacobian()
    J_det_symbolic = J_symbolic.det()

    dNdr, dNdz = shape_function_derivatives()  # dN(i)/dr, dN(i)/dz

    B = sp.zeros(4, NNODE * ND)  # B matrix with 4 rows and NNODE * ND columns

    r_iso = isoparametric_mapping()[0]

    N_funcs = shape_functions()

    for i in range(NNODE):

        # ∂Nᵢ/∂r = (1/|J|) * [ (∂z/∂Z * ∂Nᵢ/∂R) - (∂r/∂Z * ∂Nᵢ/∂Z) ]
        dNi_dr = (1/J_det_symbolic) * (sp.diff(y_map, Z) * dNdr[i] - sp.diff(x_map, Z) * dNdz[i])

        # ∂Nᵢ/∂z = (1/|J|) * [ (∂r/∂R * ∂Nᵢ/∂Z) - (∂z/∂R * ∂Nᵢ/∂R) ]
        dNi_dz = (1/J_det_symbolic) * (sp.diff(x_map, R) * dNdz[i] - sp.diff(y_map, R) * dNdr[i])

        B[0, ND*i] = dNi_dr
        B[1, ND*i] = N_funcs[i] / r_iso
        B[2, ND*i+1] = dNi_dz
        B[3, ND*i] = dNi_dz
        B[3, ND*i+1] = dNi_dr

    return B

# Example of calculating and printing the B matrix
B_matrix = calculate_B()
# You can then substitute values as needed
B_evaluated = B_matrix.subs(physical_coords_map).subs({R: 1/3, Z: 1/3})
sp.pprint(B_evaluated.evalf().applyfunc(lambda x: round(x, 3)))

⎡-3.704    0      3.704      0     7.407     0     -7.407    0   ⎤
⎢                                                                ⎥
⎢0.654     0      1.307      0     2.614     0     1.307     0   ⎥
⎢                                                                ⎥
⎢  0     -8.333     0     -16.667    0     16.667    0     8.333 ⎥
⎢                                                                ⎥
⎣-8.333  -3.704  -16.667   3.704   16.667  7.407   8.333   -7.407⎦


### The constitutive matrix $\begin{bmatrix}D\end{bmatrix}$

The constitutive matrix relates the stress vector to the strain vector. For plane stress conditions, it is given by:
$$
\begin{bmatrix}D\end{bmatrix} = \frac{E}{(1 - 2\nu)(1 + \nu)} \begin{bmatrix}
1 - \nu & \nu & \nu & 0 \\
\nu & 1 - \nu & \nu & 0 \\
\nu & \nu & 1 - \nu & 0 \\
0 & 0 & 0 & \frac{1 - 2\nu}{2
}
\end{bmatrix}
$$


In [10]:
def D_matrix():
    """
    The constitutive matrix for the axissymmetric tri element.
    """
    # Plane stress condition
    D = sp.zeros(4, 4)
    D[0, 0] = 1 - nu
    D[0, 1] = nu
    D[0, 2] = nu

    D[1, 0] = nu
    D[1, 1] = 1 - nu
    D[1, 2] = nu

    D[2, 0] = nu
    D[2, 1] = nu
    D[2, 2] = 1 - nu

    D[3, 3] = (1 - 2 * nu) / 2

    D *= E / ((1 - 2 * nu) * (1 + nu))

    return D

D = D_matrix()
sp.pprint(D.applyfunc(lambda x: round(x/1e9, 2)))
print(D.T == D)

⎡269.23  115.38  115.38    0  ⎤
⎢                             ⎥
⎢115.38  269.23  115.38    0  ⎥
⎢                             ⎥
⎢115.38  115.38  269.23    0  ⎥
⎢                             ⎥
⎣  0       0       0     76.92⎦
True


### Quadrature points and weights

In [11]:
if NNODE == 3:
    gi_data = {
        1: {
            'point': ((1/3, 1/3),),  # midpoint of the triangle
            'weight': (1,)
        },
        3: {
            'point': ((1/6, 1/6), (4/6, 1/6), (1/6, 4/6), ),
            'weight': (1/3, 1/3, 1/3)
        },
    }
elif NNODE == 4:
    gi_data = {
        1: {
            'point': [0],
            'weight': [2]
        },
        2: {
            # 'point': [-1 / np.sqrt(3), 1 / np.sqrt(3)],
            'point': [-0.5774, 0.5774],
            'weight': [1, 1]
        },
        3: {
            # 'point': [-np.sqrt(3 / 5), 0, np.sqrt(3 / 5)],
            'point': [-0.7745966, 0, 0.7745966],
            'weight': [5 / 9,  8 / 9, 5 / 9]
        },
        4: {
            'point': [-0.861136, -0.339981, 0.339981, 0.861136],
            'weight': [0.347855, 0.652145, 0.652145, 0.347855]
        },
        5: {
            'point': [-0.906180, -0.538469, 0, 0.906180, 0.538469],
            'weight': [0.236927, 0.478629, 0.568889, 0.478629, 0.236927]
        },
        6: {
            'point': [-0.932469, -0.661209, -0.238619, 0.238619, 0.661209, 0.932469],
            'weight': [0.171324, 0.360761, 0.467914, 0.467914, 0.360761, 0.171324]
        }
    }
else:
    raise ValueError("Unsupported number of nodes. Only 3 or 4 nodes are supported")

### The stiffness matrix $\begin{bmatrix}K\end{bmatrix}$

In [12]:

def K_matrix(ng) -> np.array:
    """
    Gets the stiffness matrix for the element.
    """
    K_lok = sp.zeros(NNODE * ND, NNODE * ND)  # initialize the stiffness matrix
    J = jacobian()
    J_substituted = J.subs(physical_coords_map)
    D = D_matrix()
    r_iso = isoparametric_mapping()[0]  # r(R, Z) mapping

    if NNODE == 3:
        for p, w in zip(gi_data[ng]['point'], gi_data[ng]['weight']):
            r_gp, z_gp = p[0], p[1]  # Gauss point in natural coords

            # Evaluate B, det(J), and r at the Gauss point
            B_evaluated = B_matrix.subs(physical_coords_map).subs({R: r_gp, Z: z_gp}).evalf()
            J_evaluated = J_substituted.subs({R: r_gp, Z: z_gp}).evalf()
            det_J = J_evaluated.det()
            r_physical_at_gp = r_iso.subs(physical_coords_map).subs({R: r_gp, Z: z_gp}).evalf()

            # Calculate the stiffness matrix contribution with the 2*pi*r term
            K_contribution = 2 * sp.pi * B_evaluated.T * D * B_evaluated * r_physical_at_gp * det_J * w
            K_lok += K_contribution

    elif NNODE == 4:

        for i in range(ng):
            for j in range(ng):
                s = gi_data[ng]['point'][i]
                t = gi_data[ng]['point'][j]
                w = gi_data[ng]['weight'][i] * gi_data[ng]['weight'][j]

                # Calculate the jacobian at the gauss point
                J_evaluated = J_substituted.subs({R: s, Z: t}).evalf()
                det_J = J_evaluated.det()
                r_physical_at_gp = r_iso.subs(physical_coords_map).subs({R: s, Z: t}).evalf()
                B = B_matrix.subs(physical_coords_map).subs({R: s, Z: t}).evalf()

                # Calculate the stiffness matrix contribution
                K_contribution = 2 * sp.pi * B.T * D * B * r_physical_at_gp * det_J * w
                K_lok += K_contribution

    else:
        raise ValueError("Unsupported number of nodes. Only 3 or 4 nodes are supported")

    return K_lok

ng = NGS[0]  # number of Gauss points, can be 1, 2, 3, 4, 5, or 6
K1 = K_matrix(ng)
K1_np = np.array(K1).astype(np.float64)

ng = NGS[1]
K3 = K_matrix(ng)
K3_np = np.array(K3).astype(np.float64)

print(np.round(K1_np/1e9, 2))
print()
print(np.round(K3_np/1e9, 2))


[[  66.48   38.67   15.46    1.21  -68.82  -38.67  -17.8    -1.21]
 [  38.67  155.81  -17.52  139.16  -54.98 -155.81    1.21 -139.16]
 [  15.46  -17.52   80.98  -54.98   -3.3    17.52  -68.82   54.98]
 [   1.21  139.16  -54.98  155.81  -17.52 -139.16   38.67 -155.81]
 [ -68.82  -54.98   -3.3   -17.52   80.98   54.98   15.46   17.52]
 [ -38.67 -155.81   17.52 -139.16   54.98  155.81   -1.21  139.16]
 [ -17.8     1.21  -68.82   38.67   15.46   -1.21   66.48  -38.67]
 [  -1.21 -139.16   54.98 -155.81   17.52  139.16  -38.67  155.81]]

[[  82.4    31.42   -8.53    8.46  -46.41  -42.29  -30.85    2.42]
 [  31.42  179.2   -10.27   87.23  -51.35 -103.88   -2.42 -162.56]
 [  -8.53  -10.27  116.71  -62.23  -38.16   21.15  -46.41   51.35]
 [   8.46   87.23  -62.23  236.3   -21.15 -219.65   42.29 -103.88]
 [ -46.41  -51.35  -38.16  -21.15  116.71   62.23   -8.53   10.27]
 [ -42.29 -103.88   21.15 -219.65   62.23  236.3    -8.46   87.23]
 [ -30.85   -2.42  -46.41   42.29   -8.53   -8.46   82.4   -

In [13]:
def eigen(K):
    """
    Calculate the eigenvalues and eigenvectors of the stiffness matrix K.
    """
    # Convert the stiffness matrix to a numpy array
    K_np = np.array(K).astype(np.float64)

    # the eigenvalues
    eigenvalues, eigenvectors = np.linalg.eigh(K_np)
    eigenvalues = sorted([float(x) for x in eigenvalues])
    eigenvalues = [round(ev, 5) for ev in eigenvalues]
    print('The eigenvalues of the stiffness matrix K are: {}'.format(eigenvalues))

    # print([round(ev, 5) for ev in eigenvalues])
    # print(eigenvectors.shape)
    # sp.pprint(eigenvectors)

    # # the eigenvectors for the near-zero eigenvalues
    # nearzero_eigenvalues = [ev for ev in eigenvalues if abs(ev) < 1e-10]
    # # printing the eigenvectors for the near-zero eigenvalues
    # print('The eigenvectors for the near-zero eigenvalues are:')
    # for i, ev in enumerate(nearzero_eigenvalues):
    #     print(f'Eigenvalue {i + 1}: {ev:.6f}')
    #     # print('Eigenvector:')
    #     # print(eigenvectors[:, i])
    print()


print("Eigenvalues and eigenvectors for the stiffness matrix K_gauss_1pt:")
print('one zero mode, multiple spurious modes')
eigen(K1/1e12)
print("Eigenvalues and eigenvectors for the stiffness matrix K_gauss_3pt:")
print('one zero mode, no spurious mode')
eigen(K3/1e12)

Eigenvalues and eigenvectors for the stiffness matrix K_gauss_1pt:
one zero mode, multiple spurious modes
The eigenvalues of the stiffness matrix K are: [-0.0, -0.0, 0.0, 0.0, 0.00723, 0.09179, 0.20185, 0.61729]

Eigenvalues and eigenvectors for the stiffness matrix K_gauss_3pt:
one zero mode, no spurious mode
The eigenvalues of the stiffness matrix K are: [-0.0, 0.00052, 0.00763, 0.09193, 0.09426, 0.19976, 0.20896, 0.62616]

